In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
path = 'Tabular data/'

In [3]:
dengue = pd.read_csv(path + 'dengue_tabular_v1.csv')
dengue

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,2019/w43,2019/w44,2019/w45,2019/w46,2019/w47,2019/w48,2019/w49,2019/w50,2019/w51,2019/w52
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,29,17,35,16,21,15,20,30,14,5
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,0,0,0,0,0,0,0,0,0,0
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0,0,0,0,0,0,0,0,0,0
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0,0,0,0,0,0,0,0,0,0
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,0,0,0,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,Yavaraté,555,693,841,1003,1164,1275,1349,1334,...,0,0,0,0,0,0,0,0,0,0
1117,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,0,1,2,1,1,0,0,0,1,0
1118,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,1,1,1,0,0,0,0,0,0,0
1119,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,0,0,0,0,0,0,0,0,0,0


## Preprocess new cases

In [4]:
new_cases_path = 'Tabular data/New Cases'
new_cases = []
for file in os.listdir(new_cases_path):
    if file.endswith('.xlsx'):
        new_cases.append(os.path.join(new_cases_path, file))
        
new_cases

['Tabular data/New Cases/Rutinaria_2022_Per XII.xlsx',
 'Tabular data/New Cases/rutinaria_2021.xlsx',
 'Tabular data/New Cases/rutinaria_2020.xlsx']

In [5]:
def generate_code(codes):
    
    department = codes[0]
    
    if type(department) != str:
        department = str(int(department))
    
    if len(department) == 1:
        department = '0' + department
    
    municipality = str(int(codes[1]))
    
    if department.isnumeric() and (municipality.isnumeric()):
        if len(municipality) == 3:
            code = department + municipality
        elif len(municipality) == 2:
            code = department + '0' + municipality
        elif len(municipality) == 1:
            code = department + '00' + municipality
    else:
        code = np.nan
    
    return code

In [6]:
def rename_columns(df, year):
    columns = []

    for column in df.columns:
        try:
            columns.append(f'{year}/w' + column.split("_")[1])
        except:
            columns.append(column)

    return columns

In [7]:
# Read file:
xls = pd.ExcelFile(new_cases[0])
df_2022 = pd.read_excel(xls, 'Municipio', skiprows=(8), header=0)

# Drop Unknow Municipalities
df_2022 = df_2022[df_2022['Codigo del municipio'].notna()]

# Complete the dataset
cols = ['Codigo del departamento', 'Codigo de evento', 'Nombre del evento', 'Nombre del departamento']
df_2022[cols] = df_2022[cols].fillna(method='ffill')

# Get just dengue cases
""" Codigo de evento: 210 --- Nombre del evento: DENGUE"""
df_2022 = df_2022[df_2022['Nombre del evento'] == 'DENGUE']

# Get municipality code
df_2022['Municipality code'] = df_2022[['Codigo del departamento', 'Codigo del municipio']].apply(generate_code, axis=1)

df_2022.drop(columns=['Codigo de evento', 'Nombre del evento', 'Codigo del departamento', 'Nombre del departamento', 'Codigo del municipio', 'Nombre del municipio', 'Total general'], inplace=True)

# NaN -> 0
df_2022 = df_2022.fillna(0)

df_2022.columns = rename_columns(df=df_2022, year='2022')

cols = df_2022.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_2022 = df_2022[cols]


df_2022

,Municipality code,2022/w1,2022/w2,2022/w3,2022/w4,2022/w5,2022/w6,2022/w7,2022/w8,2022/w9,...,2022/w40,2022/w41,2022/w42,2022/w43,2022/w44,2022/w45,2022/w46,2022/w47,2022/w48,2022/w49
3295,91001,7.0,9.0,8.0,11.0,11.0,9.0,7.0,8.0,12.0,...,5.0,4.0,4.0,16.0,16.0,17.0,6.0,8.0,3.0,0.0
3296,91263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3297,91798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3298,91540,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3299,91407,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,52696,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0
4077,52352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4078,52390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4079,52786,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [8]:
# Read file:
xls = pd.ExcelFile(new_cases[1])
df_2021 = pd.read_excel(xls, 'Municipio', skiprows=(3), header=0)

# Complete the dataset
cols = ['COD_DPTO_O', 'Departamento_Ocurrencia', 'COD_MUN_O', 'Municipio_Ocurrencia']
df_2021[cols] = df_2021[cols].fillna(method='ffill')

# Get just dengue cases
""" Codigo de evento: 210 --- Nombre del evento: DENGUE"""
df_2021 = df_2021[df_2021['evento'] == 'DENGUE']

# Get municipality code
df_2021['Municipality code'] = df_2021[['COD_DPTO_O', 'COD_MUN_O']].apply(generate_code, axis=1)

df_2021.drop(columns=['COD_DPTO_O', 'Departamento_Ocurrencia', 'COD_MUN_O', 'Municipio_Ocurrencia', 'COD_EVE', 'evento', 'Total general'], inplace=True)

df_2021.columns = rename_columns(df=df_2021, year='2021')

## Sum week 52 and 53:
df_2021['2021/w52'] = df_2021['2021/w52'] + df_2021['2021/w53']
df_2021.drop(columns='2021/w53', inplace=True)

cols = df_2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_2021 = df_2021[cols]


df_2021

,Municipality code,2021/w1,2021/w2,2021/w3,2021/w4,2021/w5,2021/w6,2021/w7,2021/w8,2021/w9,...,2021/w43,2021/w44,2021/w45,2021/w46,2021/w47,2021/w48,2021/w49,2021/w50,2021/w51,2021/w52
5,05001,4.0,7.0,5.0,6.0,7.0,7.0,4.0,5.0,4.0,...,6.0,4.0,7.0,13.0,7.0,3.0,6.0,5.0,5.0,2.0
93,05030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
119,05031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207,05040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229,05042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22823,97001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22910,99001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
22942,99524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22964,99624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Read file:
xls = pd.ExcelFile(new_cases[2])
df_2020 = pd.read_excel(xls, 'Por municipios', skiprows=(5), header=0)

# Complete the dataset
cols = ['COD_DPTO_O', 'NDEP_PROCE', 'COD_MUN_O', 'NMUN_PROCE']
df_2020[cols] = df_2020[cols].fillna(method='ffill')

# Get just dengue cases
""" Codigo de evento: 210 --- Nombre del evento: DENGUE"""
df_2020 = df_2020[df_2020['NOM_EVE'] == 'DENGUE']

# Get municipality code
df_2020['Municipality code'] = df_2020[['COD_DPTO_O', 'COD_MUN_O']].apply(generate_code, axis=1)

df_2020.drop(columns=['COD_DPTO_O', 'NDEP_PROCE', 'COD_MUN_O', 'NMUN_PROCE', 'COD_EVE', 'NOM_EVE', 'Total general'], inplace=True)

df_2020.columns = rename_columns(df=df_2020, year='2020')

cols = df_2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_2020 = df_2020[cols]


df_2020

,Municipality code,2020/w01,2020/w02,2020/w03,2020/w04,2020/w05,2020/w06,2020/w07,2020/w08,2020/w09,...,2020/w44,2020/w45,2020/w46,2020/w47,2020/w48,2020/w49,2020/w50,2020/w51,2020/w52,2020/w53
12,05001,35,43,37,23,20,36,33,21,21,...,6,3,5,10,5,7,7,5,5,1
67,05002,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,05004,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,05030,1,0,3,0,0,1,1,2,0,...,0,0,0,0,1,0,2,0,0,1
150,05031,0,0,0,0,0,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24471,97161,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24538,99001,0,0,0,0,0,0,0,0,0,...,0,2,2,0,0,0,0,0,0,0
24574,99524,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24602,99624,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Merge

In [10]:
df_2020_2021 = df_2020.merge(df_2021, on='Municipality code', how='outer').fillna(0)
new_dengue_df = df_2020_2021.merge(df_2022, on='Municipality code', how='outer').fillna(0)
new_dengue_df['Municipality code'] = new_dengue_df['Municipality code'].astype(int)
new_dengue_df

,Municipality code,2020/w01,2020/w02,2020/w03,2020/w04,2020/w05,2020/w06,2020/w07,2020/w08,2020/w09,...,2022/w40,2022/w41,2022/w42,2022/w43,2022/w44,2022/w45,2022/w46,2022/w47,2022/w48,2022/w49
0,5001,35.0,43.0,37.0,23.0,20.0,36.0,33.0,21.0,21.0,...,8.0,4.0,8.0,6.0,7.0,5.0,10.0,7.0,1.0,0.0
1,5002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5030,1.0,0.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,73024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
932,97511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
933,52215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
934,52418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Merge All

In [11]:
final_df = dengue.merge(new_dengue_df, on='Municipality code', how='left')
cols = final_df.isna().columns[final_df.isna().sum() > 1].tolist()

final_df[cols] = final_df[cols].fillna(0)

final_df

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,2022/w40,2022/w41,2022/w42,2022/w43,2022/w44,2022/w45,2022/w46,2022/w47,2022/w48,2022/w49
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,8.0,4.0,8.0,6.0,7.0,5.0,10.0,7.0,1.0,0.0
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,Yavaraté,555,693,841,1003,1164,1275,1349,1334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1117,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1118,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1119,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save csv file

In [12]:
final_df.to_csv(path + 'dengue_tabular.csv', index=False)

In [13]:
pd.read_csv(path+'dengue_tabular.csv')

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,2022/w40,2022/w41,2022/w42,2022/w43,2022/w44,2022/w45,2022/w46,2022/w47,2022/w48,2022/w49
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,8.0,4.0,8.0,6.0,7.0,5.0,10.0,7.0,1.0,0.0
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,Yavaraté,555,693,841,1003,1164,1275,1349,1334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1117,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1118,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1119,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
